In [ ]:
# Dynamo DB 생성
# aws -> Dynamo DB -> create table

<img src="./imgs/2_3.png" width="60%">

<img src="./imgs/2_2.png" width="60%">

In [ ]:
# primary key(Partition key)를 하나만 가져오면 track이 여러개일 경우 다 가져오기가 힘들다.
# add sort key, id로 하나 더 만들어주자.

In [2]:
#boto3는 aws가 제공하는 python sdk이다.

In [8]:
import sys
import os
import boto3
import requests
import base64
import json
import logging
import pymysql
import pickle

file = open("/Users/kangyeolyoun/Desktop/pickle file/aws_mysql_db", "rb")
secret_data = pickle.load(file)

host = secret_data[0]
port = int(secret_data[1])
username = "kangyeol"
database = secret_data[2]
password = secret_data[3]
client_id = secret_data[4]
client_secret = secret_data[5]

def get_headers(client_id, client_secret):

    endpoint = "https://accounts.spotify.com/api/token"
    encoded = base64.b64encode("{}:{}".format(client_id, client_secret).encode('utf-8')).decode('ascii')

    headers = {
        "Authorization": "Basic {}".format(encoded)
    }

    payload = {
        "grant_type": "client_credentials"
    }

    r = requests.post(endpoint, data=payload, headers=headers)

    access_token = json.loads(r.text)['access_token']

    headers = {
        "Authorization": "Bearer {}".format(access_token)
    }

    return headers



def main():    
    
    try:
        dynamodb = boto3.resource("dynamodb", region_name="ap-northeast-2",\
                                 endpoint_url="http://dynamodb.ap-northeast-2.amazonaws.com")
    
    except:
        logging.error("could not connect to dynamodb")
        sys.exit(1)
        
        
    try:
        conn = pymysql.connect(host, user=username, passwd=password, db=database,\
                         port=port, use_unicode=True, charset="utf8")
        cursor = conn.cursor()
    except:
        logging.error("could not connect to RDS")
        sys.exit(1)
        
    
    headers = get_headers(client_id, client_secret)
    
    #table을 만들었으니 테이블 지정해줍시다.
    table = dynamodb.Table('top_tracks')
    
    cursor.execute("SELECT id FROM artists LIMIT 1")
    
    for (artist_id, ) in cursor.fetchall():
        
        URL = "https://api.spotify.com/v1/artists/{}/top-tracks".format(artist_id)
        params = {
            "country": 'US'
        }
        
        r = requests.get(URL, params=params, headers=headers)
        
        raw = json.loads(r.text)
        
        for track in raw["tracks"]:
            data = {
                "artist_id":artist_id
            }
            
            data.update(track)
            table.put_item(
                Item=data
            )
    
    
if __name__ == "__main__":
    main()

### batch 형식으로 insert 해봅시다.

In [ ]:
import sys
import os
import boto3
import requests
import base64
import json
import logging
import pymysql
import pickle

file = open("/Users/kangyeolyoun/Desktop/pickle file/aws_mysql_db", "rb")
secret_data = pickle.load(file)

host = secret_data[0]
port = int(secret_data[1])
username = "kangyeol"
database = secret_data[2]
password = secret_data[3]
client_id = secret_data[4]
client_secret = secret_data[5]

def get_headers(client_id, client_secret):

    endpoint = "https://accounts.spotify.com/api/token"
    encoded = base64.b64encode("{}:{}".format(client_id, client_secret).encode('utf-8')).decode('ascii')

    headers = {
        "Authorization": "Basic {}".format(encoded)
    }

    payload = {
        "grant_type": "client_credentials"
    }

    r = requests.post(endpoint, data=payload, headers=headers)

    access_token = json.loads(r.text)['access_token']

    headers = {
        "Authorization": "Bearer {}".format(access_token)
    }

    return headers



def main():    
    
    try:
        dynamodb = boto3.resource("dynamodb", region_name="ap-northeast-2",\
                                 endpoint_url="http://dynamodb.ap-northeast-2.amazonaws.com")
    
    except:
        logging.error("could not connect to dynamodb")
        sys.exit(1)
        
        
    try:
        conn = pymysql.connect(host, user=username, passwd=password, db=database,\
                         port=port, use_unicode=True, charset="utf8")
        cursor = conn.cursor()
    except:
        logging.error("could not connect to RDS")
        sys.exit(1)
        
    
    headers = get_headers(client_id, client_secret)
    
    #table을 만들었으니 테이블 지정해줍시다.
    table = dynamodb.Table('top_tracks')
    
    cursor.execute("SELECT id FROM artists LIMIT 1")
    
    for (artist_id, ) in cursor.fetchall():
        
        URL = "https://api.spotify.com/v1/artists/{}/top-tracks".format(artist_id)
        params = {
            "country": 'US'
        }
        
        r = requests.get(URL, params=params, headers=headers)
        
        raw = json.loads(r.text)
        
        for track in raw["tracks"]:
            data = {
                "artist_id":artist_id
            }
            
            data.update(track)
            table.put_item(
                Item=data
            )
    
    
if __name__ == "__main__":
    main()

In [ ]:
### get items해봅시다.

In [10]:

def main():    
    
    try:
        dynamodb = boto3.resource("dynamodb", region_name="ap-northeast-2",\
                                 endpoint_url="http://dynamodb.ap-northeast-2.amazonaws.com")
    
    except:
        logging.error("could not connect to dynamodb")
        sys.exit(1)
        
        
    table = dynamodb.Table("top_tracks")
    
    response = table.get_item(
        Key = {
            "artist_id": "00FQb4jTyendYWaN8pK0wa",
            "id": "0Oqc0kKFsQ6MhFOLBNZIGX"
        }
    
    )
    
    print(response)
    
if __name__ == "__main__":
    main()

{'Item': {'is_playable': True, 'duration_ms': Decimal('202192'), 'external_ids': {'isrc': 'GBUM71903894'}, 'uri': 'spotify:track:0Oqc0kKFsQ6MhFOLBNZIGX', 'name': "Doin' Time", 'album': {'total_tracks': Decimal('14'), 'images': [{'width': Decimal('640'), 'url': 'https://i.scdn.co/image/ab67616d0000b273db58320165892f952a6ddb3f', 'height': Decimal('640')}, {'width': Decimal('300'), 'url': 'https://i.scdn.co/image/ab67616d00001e02db58320165892f952a6ddb3f', 'height': Decimal('300')}, {'width': Decimal('64'), 'url': 'https://i.scdn.co/image/ab67616d00004851db58320165892f952a6ddb3f', 'height': Decimal('64')}], 'artists': [{'name': 'Lana Del Rey', 'href': 'https://api.spotify.com/v1/artists/00FQb4jTyendYWaN8pK0wa', 'id': '00FQb4jTyendYWaN8pK0wa', 'type': 'artist', 'external_urls': {'spotify': 'https://open.spotify.com/artist/00FQb4jTyendYWaN8pK0wa'}, 'uri': 'spotify:artist:00FQb4jTyendYWaN8pK0wa'}], 'release_date': '2019-08-30', 'name': 'Norman Fucking Rockwell!', 'album_type': 'album', 'relea

In [17]:

from boto3.dynamodb.conditions import Key, Attr

def main():    
    
    try:
        dynamodb = boto3.resource("dynamodb", region_name="ap-northeast-2",\
                                 endpoint_url="http://dynamodb.ap-northeast-2.amazonaws.com")
    
    except:
        logging.error("could not connect to dynamodb")
        sys.exit(1)
        
        
    table = dynamodb.Table("top_tracks")
    
    
    #Query, Scanning 위의 Key에서 1개의 값만 알경우 이 옵션을 사용하면 좋다.
    #id모르니까 다 가져와줘~
    response = table.query(
        KeyConditionExpression=Key('artist_id').eq('00FQb4jTyendYWaN8pK0wa'),
        #popularity를 이용해서도 가능
        FilterExpression=Attr("popularity").gt(80)
    )
    
    print(response["Items"])
    print(len(response["Items"]))
    
if __name__ == "__main__":
    main()

[{'is_playable': True, 'duration_ms': Decimal('190066'), 'external_ids': {'isrc': 'USUM71912501'}, 'uri': 'spotify:track:6zegtH6XXd2PDPLvy1Y0n2', 'name': 'Don’t Call Me Angel (Charlie’s Angels) (with Miley Cyrus & Lana Del Rey)', 'album': {'total_tracks': Decimal('11'), 'images': [{'width': Decimal('640'), 'url': 'https://i.scdn.co/image/8c1643c6a03f950c193184e1e35de3969d311adc', 'height': Decimal('640')}, {'width': Decimal('300'), 'url': 'https://i.scdn.co/image/eb1b71980f84221c6f602fd21cec6487967b0ebe', 'height': Decimal('300')}, {'width': Decimal('64'), 'url': 'https://i.scdn.co/image/b12875878a88dbc47b875d0a62119228afcda219', 'height': Decimal('64')}], 'artists': [{'name': 'Various Artists', 'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of', 'id': '0LyfQWJT6nXafLPZqxe9Of', 'type': 'artist', 'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'}, 'uri': 'spotify:artist:0LyfQWJT6nXafLPZqxe9Of'}], 'release_date': '2019-11-01', 'name': 